# Cycle Dehaze

## Build Model

In [1]:
import sys
sys.path.append('../')

from cycle_dehaze import CycleDehaze
from data_loader.reside_sample_data_loader import RESIDEOTSSampleDataLoader as DataLoader
import numpy as np
from skimage.measure import compare_psnr, compare_ssim
import tensorflow as tf

model = CycleDehaze()
dl = DataLoader()

## Train Model

In [2]:
datasetX, datasetY = dl.train_data()
model.train(30, datasetX, datasetY, 8, 10000)

epoch 1/30
Loss: Dx: 0.8642263412475586 Dy: 0.8101974725723267 G: 24.68951416015625 F: 24.729290008544922
Time taken for epoch 1 of totoal epoch 30 is 0:03:17.877875

epoch 2/30
Loss: Dx: 0.6853303909301758 Dy: 0.7010290622711182 G: 4.8360724449157715 F: 4.150176525115967
Time taken for epoch 2 of totoal epoch 30 is 0:02:05.947655

epoch 3/30
Loss: Dx: 0.6726367473602295 Dy: 0.3775876760482788 G: 4.805303573608398 F: 4.392053127288818
Time taken for epoch 3 of totoal epoch 30 is 0:02:06.359306

epoch 4/30
Loss: Dx: 0.39749738574028015 Dy: 0.137455552816391 G: 5.948309898376465 F: 4.871628761291504
Time taken for epoch 4 of totoal epoch 30 is 0:02:06.644752

epoch 5/30
Loss: Dx: 0.26103657484054565 Dy: 0.33430689573287964 G: 4.761781215667725 F: 4.908614635467529
Saving checkpoint for epoch 5 at ./checkpoints/train\ckpt-1
Time taken for epoch 5 of totoal epoch 30 is 0:02:10.814346

epoch 6/30
Loss: Dx: 0.22008313238620758 Dy: 0.1918846070766449 G: 5.508143424987793 F: 5.513810634613037


## Evaluate

In [3]:
def evaluate(model, datasetX, datasetY, batch_size):

    psnr_list, ssim_list = [], []
    
    for image_x, image_y in tf.data.Dataset.zip((datasetX, datasetY)).batch(batch_size):
        fake_y = model.predict(image_x)
        for i in range(image_y.shape[0]):
            psnr_list.append(compare_psnr(image_y.numpy()[i], fake_y[i], data_range=1))
            ssim_list.append(compare_ssim(image_y.numpy()[i], fake_y[i], 3))

    return psnr_list, ssim_list

datasetX, datasetY = dl.test_data()
psnr_list, ssim_list = evaluate(model.generator_f, datasetX, datasetY, 8)

print(np.mean(psnr_list), np.mean(ssim_list))

13.648856525113775 0.6249173335733427
